## 学習済みモデルの読み込みと評価データによる評価

### データ・モデルの読み込み（学習・訓練時と同様）

 - データは極性判定コーパスの一部を利用 <br />
     - 学習・開発データとは別データ
 - モデルは多層パーセプトロン（MLP）を利用 <br />
     - モデルのインスタンスを作って，そのモデルに学習済みパラメータを読み込んで設定する
 - 文章をBoWで表現し，MLPで分類
 

In [5]:
import random

from tqdm import tqdm
import MeCab
import torch
from torch import optim

from data import Sentence
from data import DataSet
from data import DataLoader
from mlp import MultiLayer # 多層パーセプトロンを利用

mecab_parser = MeCab.Tagger("-Owakati -d /usr/local/lib/mecab/dic/mecab-ipadic-neologd")
mecab_parser.parse("")

'\n'

In [6]:
def load_document(path):
    """
    データ読み込み用関数
    Args:
        path:対象ファイルのパス
    Return:
        tag_list (List[string])  :一行ごとのタグを要素に持つリスト
        sentences_list (List[string]):一行ごとの文を要素に持つリスト
    """
    with open(path) as f:
        texts = f.readlines()

    sentences_list,tag_list = [],[]
    for row in texts:
        sentence_list = []
        tag,words = map(str,row.split("\t"))
        tag_list.append(tag)
        node = mecab_parser.parseToNode(words.replace('\n',''))
        while node:
            analysis = node.feature.split(',')
            if analysis[0] != 'BOS/EOS':
                if analysis[0] == '名詞' and node.surface != '':
                    sentence_list.append(node.surface)
                elif analysis[6] != '':
                    sentence_list.append(analysis[6])
            node = node.next
        sentences_list.append(sentence_list)
    return tag_list, sentences_list

In [7]:
# testデータのパス
test_path = "test_data.txt"

# 文書の読み込み
test_labels, test = load_document(test_path)
test_data = DataSet(test, vocab=vocab, label_dict=label_to_id_dict)
test_data_size = len(test_data)

In [8]:
# 訓練時の語彙リストの読み込み (BoW作成のため)
vocab = [l.strip() for l in open("vocab.txt")]
# ラベル辞書の読み込み（学習時に出力層のどの次元がどのクラスと対応していたか判別するため）
label_to_id_dict = {}
id_to_label_dict = {}
for l in open("label_dict.tsv"):
    label_name, label_id = l.strip().split("\t")
    label_to_id_dict[label_name] = int(label_id)
    id_to_label_dict[int(label_id)] = label_name

### 学習済みモデルの読み込み

In [9]:
# モデルのインスタンスを作成（初期化）
multi_layer_model = MultiLayer(len(vocab), len(label_to_id_dict))
# パラメータをモデルインスタンスにロード
multi_layer_model.load_state_dict(torch.load("output_models/best_model"))

<All keys matched successfully>

### 評価データでの評価

In [10]:
# testデータでの評価
multi_layer_model.eval()
test_data_loader = DataLoader(test_data, batch_size=10, shuffle=False)
predicted_label_id_list = multi_layer_model.test(test_data_loader) # モデルから予測結果のidが出力される

predicted_label_list = [id_to_label_dict[idx.item()] for idx in predicted_label_id_list] # idをラベル名に直す

In [11]:
from sklearn.metrics import classification_report
report = str(classification_report(test_labels, predicted_label_list, digits=4))
print(report)


              precision    recall  f1-score   support

          +1     0.6827    0.7100    0.6961       100
          -1     0.6979    0.6700    0.6837       100

    accuracy                         0.6900       200
   macro avg     0.6903    0.6900    0.6899       200
weighted avg     0.6903    0.6900    0.6899       200

